In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom

### ASSUMPTIONS:
- Velocity Models
- Receiver Geom

Already made, therefore only need to set the path.

In [2]:
project_dir = '/home/birniece/Documents/Projects/MicroseismicModelling/MicroseismicProcessing/FDModelling/UTAH_Homo_multi'
expname = 'UTAH_Homo_tst'

model_dir = '/home/birniece/Documents/data/UTAH_FORGE/ClairesVMods/homo/UtahFORGE_homo_5m'
rec_file = '/home/birniece/Documents/data/UTAH_FORGE/ReceiverLocations/grid_receiver.dat'
src_file = '/home/birniece/Documents/data/UTAH_FORGE/SourceFiles/multishot_sources.dat'

os.makedirs(project_dir, exist_ok=True)  # make project dir if doesn't already exist

# SELECT PARAMETERS

### MODEL PARAMETERS - Get these from the script that made the models

In [3]:
# TIME PARAMETERS
dt = 0.0005
tdur = 1.0

In [4]:
# SPATIAL PARAMETERS
dx = dy = dz = 5.

xmax = 1700
ymax = 1740 # y is y, I fix the SOFI ordering later
zmax = 1500

nx_homo = int(xmax//dx)
ny_homo = int(ymax//dy)
nz_homo = int(zmax//dz)

n_xzy = [nx_homo, nz_homo, ny_homo]
d_xzy = [dx, dz, dy]

In [5]:
# BOUNDARIES & FREE SURFACE
nbounds = 30
free_surface = True

### SOURCE DEFINITION

In [6]:
grid_sx_locs = np.arange(750, 950, 15)
grid_sy_locs = np.arange(770, 970, 15)
grid_sz_locs = np.arange(1100, 1300, 15)

In [7]:
sx, sy, sz = np.meshgrid(grid_sx_locs,
                         grid_sy_locs,
                         grid_sz_locs) 
srclocs = np.vstack((sx.flatten(), sy.flatten(), sz.flatten()))# y is y, I fix the SOFI ordering later
srclocs.shape

(3, 2744)

In [8]:
src_xyz_list = []
for srcloc in srclocs.T:
    src_xyz_list.append(srcloc)
src_xyz_list

[array([ 750,  770, 1100]),
 array([ 750,  770, 1115]),
 array([ 750,  770, 1130]),
 array([ 750,  770, 1145]),
 array([ 750,  770, 1160]),
 array([ 750,  770, 1175]),
 array([ 750,  770, 1190]),
 array([ 750,  770, 1205]),
 array([ 750,  770, 1220]),
 array([ 750,  770, 1235]),
 array([ 750,  770, 1250]),
 array([ 750,  770, 1265]),
 array([ 750,  770, 1280]),
 array([ 750,  770, 1295]),
 array([ 765,  770, 1100]),
 array([ 765,  770, 1115]),
 array([ 765,  770, 1130]),
 array([ 765,  770, 1145]),
 array([ 765,  770, 1160]),
 array([ 765,  770, 1175]),
 array([ 765,  770, 1190]),
 array([ 765,  770, 1205]),
 array([ 765,  770, 1220]),
 array([ 765,  770, 1235]),
 array([ 765,  770, 1250]),
 array([ 765,  770, 1265]),
 array([ 765,  770, 1280]),
 array([ 765,  770, 1295]),
 array([ 780,  770, 1100]),
 array([ 780,  770, 1115]),
 array([ 780,  770, 1130]),
 array([ 780,  770, 1145]),
 array([ 780,  770, 1160]),
 array([ 780,  770, 1175]),
 array([ 780,  770, 1190]),
 array([ 780,  770, 

In [9]:
# src_xyz_list = [[850, 860, 1200],
#               [420, 860, 1200]]  
td = dt  # ignition time
fc = 20  # central frequency
amp = 1.  # amplitude

string_list = []
for src in src_xyz_list:
    string_list.append([str(i) for i in [src[0], src[2], src[1], td, fc, amp]])

In [10]:
string_list

[['750', '1100', '770', '0.0005', '20', '1.0'],
 ['750', '1115', '770', '0.0005', '20', '1.0'],
 ['750', '1130', '770', '0.0005', '20', '1.0'],
 ['750', '1145', '770', '0.0005', '20', '1.0'],
 ['750', '1160', '770', '0.0005', '20', '1.0'],
 ['750', '1175', '770', '0.0005', '20', '1.0'],
 ['750', '1190', '770', '0.0005', '20', '1.0'],
 ['750', '1205', '770', '0.0005', '20', '1.0'],
 ['750', '1220', '770', '0.0005', '20', '1.0'],
 ['750', '1235', '770', '0.0005', '20', '1.0'],
 ['750', '1250', '770', '0.0005', '20', '1.0'],
 ['750', '1265', '770', '0.0005', '20', '1.0'],
 ['750', '1280', '770', '0.0005', '20', '1.0'],
 ['750', '1295', '770', '0.0005', '20', '1.0'],
 ['765', '1100', '770', '0.0005', '20', '1.0'],
 ['765', '1115', '770', '0.0005', '20', '1.0'],
 ['765', '1130', '770', '0.0005', '20', '1.0'],
 ['765', '1145', '770', '0.0005', '20', '1.0'],
 ['765', '1160', '770', '0.0005', '20', '1.0'],
 ['765', '1175', '770', '0.0005', '20', '1.0'],
 ['765', '1190', '770', '0.0005', '20', 

### RECORDING PARAMETERS

In [11]:
# SNAP PARAMETERS
snap_start = dt
snap_end = tdur/2
snap_step = 100*dt
tsnap_params = [snap_start, snap_end, snap_step]
snap_sbsmp_xyz = 1

In [12]:
# SEISMOGRAM PARAMETERS
smgrm_ndt = 2

### SOFI PARAMETERS

In [13]:
sofimaster = '~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/'
num_procs = 8

# WRITE PARAMETERS TO FILE

In [14]:
# SOURCE FILE
text_file = open(src_file, "wt")
for string_line in string_list:
    text_file.writelines('\t'.join(string_line)+'\n')
text_file.close()

In [15]:
# FILE NAMES
SOFIjsonfilename = os.path.join(project_dir,'SOFI3D_%s.json'%expname)
sofi_param_file = "SOFI3D_%s.json"%expname
shellfilename = os.path.join(project_dir,'runSOFI3D_%s.sh'%expname)

In [16]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings(rec_file=rec_file)

# MODELLING
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xzy, 
                              d_xzy,
                              expname,
                              moddir=model_dir,
                             )
bndry_str = get_boundary_str(fs=free_surface, 
                             npad=nbounds, 
                             cpml=False)

# SNAP & SEISMOGRAM
monitor_strs = get_monitor_str(tsnap_params, 
                               smgrm_ndt, 
                               expname, 
                               sbsmp_xyz=snap_sbsmp_xyz,
                               snap=False
                              )


# SOURCE
src_str = get_source_str(sfile=src_file, multisource=1)
# src_str = get_MTsource_str(strike, dip, rake, sfile=source_file)

In [17]:
write_SOFIjsonParams = write_SOFIjsonParams(default_strs, 
                                            monitor_strs,  
                                            t_str, 
                                            mod_strs, 
                                            src_str, 
                                            bndry_str, 
                                            SOFIjsonfilename)
runscript = write_SOFIrunScript_NoSnaps(sofimaster, 
                                        num_procs, 
                                        sofi_param_file, 
                                        shellfilename)